In [ ]:
pip install -q tensorflow sklearn numpy

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import reuters

In [ ]:
vocab_size = 10000
max_length = 200
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=vocab_size)


2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train_padded = pad_sequences(x_train, maxlen=max_length, padding='post')
x_test_padded = pad_sequences(x_test, maxlen=max_length, padding='post')

In [ ]:
num_classes = np.max(y_train) + 1
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)


In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    LSTM(64),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 👉 Esto construye el modelo antes del resumen
model(np.zeros((1, max_length), dtype=int))

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (1, 200, 64)           │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (1, 64)                │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (1, 64)                │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (1, 64)                │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (1, 46)                │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 680,174 (2.59 MB)

 Trainable params: 680,174 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(x_train_padded, y_train_cat, epochs=5, batch_size=64, validation_data=(x_test_padded, y_test_cat))


Epoch 1/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 28s 182ms/step - accuracy: 0.3061 - loss: 2.9471 - val_accuracy: 0.3700 - val_loss: 2.3826
Epoch 2/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 39s 165ms/step - accuracy: 0.3517 - loss: 2.4165 - val_accuracy: 0.3700 - val_loss: 2.3514
Epoch 3/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 41s 168ms/step - accuracy: 0.3693 - loss: 2.3229 - val_accuracy: 0.3847 - val_loss: 2.3010
Epoch 4/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 42s 176ms/step - accuracy: 0.3720 - loss: 2.3109 - val_accuracy: 0.3709 - val_loss: 2.3981
Epoch 5/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 40s 171ms/step - accuracy: 0.3613 - loss: 2.3692 - val_accuracy: 0.3727 - val_loss: 2.3139


In [ ]:
loss, acc = model.evaluate(x_test_padded, y_test_cat)
print(f"\nPrecisión del modelo en el set de prueba: {acc:.2f}")


71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.3739 - loss: 2.2919

Precisión del modelo en el set de prueba: 0.37


In [ ]:
diccionario = reuters.get_word_index()
inverso = {value + 3: key for key, value in diccionario.items()}
inverso[0] = '<PAD>'
inverso[1] = '<START>'
inverso[2] = '<UNK>'
inverso[3] = '<UNUSED>'

def decodificar_texto(seq):
    return ' '.join([inverso.get(i, '?') for i in seq])

print("\nEjemplo de texto reconstruido:")
print(decodificar_texto(x_test[0]))
print("\nEtiqueta real:", y_test[0])


Ejemplo de texto reconstruido:
<START> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to <UNK> growth and expansion plans for <UNK> inc and <UNK> inc over the next two years a and p said the acquisition of <UNK> in august 1986 and <UNK> in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt <UNK> <UNK> of west germany reuter 3

Etiqueta real: 3


In [ ]:
pred = model.predict(np.expand_dims(x_test_padded[0], axis=0))
print("Etiqueta predicha:", np.argmax(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Etiqueta predicha: 3
